In [1]:
# Let's import all required libraries
import selenium                                  #library that is used to work with selenium
from selenium import webdriver                   #importing webdriver module from selenium to open automated chrome window
from bs4 import BeautifulSoup
import requests
import pandas as pd                              #to create DataFrame
import numpy as np
from selenium.webdriver.common.by import By      #importing inbuilt class By
import time
import re
# Importing required Exceptions which needs to be handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
'''
1. ScrapethedetailsofmostviewedvideosonYouTubefromWikipedia.
Url = https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos
You need to find following details: 
A) Rank
B) Name
C) Artist
D) Upload date 
E) Views
'''

# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up wikipedia website on automated chrome window
url = 'https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos'
driver.get(url)

time.sleep(1)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
    
    def get_results(self):
        
        page = self.driver.find_elements(By.XPATH,'//table[@class="wikitable sortable jquery-tablesorter"][1]')
        time.sleep(3)
          
        columns = ['Rank', 'Name', 'Artist', 'Views', 'Upload Date']
        data = {}
        time.sleep(2)

        elements = self.driver.find_elements(By.XPATH,'//table[@class="wikitable sortable jquery-tablesorter"][1]/tbody/tr')
        for element in elements:
            current_index = elements.index(element) + 1
            parsed_data = []
            for column in columns:
                value = element.find_element(By.XPATH,'//table[@class="wikitable sortable jquery-tablesorter"][1]/tbody/tr[{}]/td[{}]'.format(current_index, columns.index(column) + 1)).text
                parsed_data.append(value)
            data.update({current_index: parsed_data})
        data = pd.DataFrame(data.values())
        data.columns = columns
        return data


if "__main__" == __name__:
    
    table = Table(driver)
    most_viewed_videos = table.get_results()
    most_viewed_videos['Name'] = most_viewed_videos['Name'].map(lambda x: x.lstrip('"').rstrip('[1234567890"]'))

In [3]:
most_viewed_videos

,Rank,Name,Artist,Views,Upload Date
0,1.,Baby Shark Dance,Pinkfong Baby Shark - Kids' Songs & Stories,11.83,"June 17, 2016"
1,2.,Despacito,Luis Fonsi,8.02,"January 12, 2017"
2,3.,Johny Johny Yes Papa,LooLoo Kids,6.54,"October 8, 2016"
3,4.,Shape of You,Ed Sheeran,5.86,"January 30, 2017"
4,5.,Bath Song,Cocomelon – Nursery Rhymes,5.81,"May 2, 2018"
5,6.,See You Again,Wiz Khalifa,5.71,"April 6, 2015"
6,7.,Phonics Song with Two Words,ChuChu TV,5.04,"March 6, 2014"
7,8.,Uptown Funk,Mark Ronson,4.77,"November 19, 2014"
8,9.,Learning Colors – Colorful Eggs on a Farm,Miroshka TV,4.74,"February 27, 2018"
9,10.,Wheels on the Bus,Cocomelon – Nursery Rhymes,4.69,"May 24, 2018"


In [4]:
'''
2. Scrape the details team India’s international fixtures from bcci.tv. Url = https://www.bcci.tv/.
You need to find following details:
A) Match title (I.e. 1st ODI)
B) Series
C) Place
D) Date
E) Time
Note: - From bcci.tv home page you have reach to the international fixture page through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up bcci.tv website on automated chrome window
url = 'https://www.bcci.tv/'
driver.get(url)

# clicking on INTERNATIONAL to go on international fixtures page
search_btn = driver.find_element(By.XPATH, "//a[normalize-space()='INTERNATIONAL']")
search_btn.click()
time.sleep(5)

# Bring More fixtures to view to click on it
driver.execute_script("window.scrollBy(0,1500)")

# clicking on more fixtures to load all fixtures
search_btn = driver.find_element(By.XPATH, "//button[normalize-space()='More Fixtures']")
search_btn.click()
time.sleep(2)

# initializing required empty list
match_title = []
series = []
place = []
date = []
time_ = []
urls = []

url_tag = driver.find_elements(By.XPATH,'//div[@class="fixture-card-bottom"]/a')
for i in url_tag:
    urls.append(i.get_attribute('href'))

# for storing match title:
match_title_tags = driver.find_elements(By.XPATH,"//div[@class='fixture-card-bottom']/div/span[1]")
for i in match_title_tags:
    match_title.append(i.text.rstrip("- "))

# for storing match series:
series_tags = driver.find_elements(By.XPATH, "//div[@class='tab-inner-content']/div[@class='fixture-tab-inner row']/div//h5[2]/span")
for i in series_tags:
    series.append(i.text)
    
# for storing match date:
date_tags = driver.find_elements(By.XPATH,'//div[@class="match-top-info d-flex align-items-center justify-content-between"]/div[1]')
for i in date_tags:
    date.append(i.text.replace('\n',' '))
    
# for storing match time:
time_tags = driver.find_elements(By.XPATH,'//div[@class="match-top-info d-flex align-items-center justify-content-between"]/div[2]')
for i in time_tags:
    time_.append(i.text.replace('\n',' '))

for u in urls:
    driver.get(u)
    time.sleep(2)
    # for storing match location:
    place_tags = driver.find_element(By.XPATH,"//div[@class='matchVenue alignC']")
    place.append(place_tags.text)

international_fixtures = pd.DataFrame({'Match title':match_title, 'Series':series, 'Place':place, 'Date':date, 'Time':time_})
international_fixtures

,Match title,Series,Place,Date,Time
0,2nd T20I,AUSTRALIA WOMEN TOUR OF INDIA 2022,"DY PATIL STADIUM , NAVI MUMBAI",11 DEC 2022,7:00 PM IST
1,1st Test,INDIA TOUR OF BANGLADESH TEST SERIES 2022-23,"ZAHUR AHMED CHOWDHURY STADIUM , CHATTOGRAM",14 DEC 2022,9:30 AM IST
2,3rd T20I,AUSTRALIA WOMEN TOUR OF INDIA 2022,"BRABOURNE - CCI , MUMBAI",14 DEC 2022,7:00 PM IST
3,4th T20I,AUSTRALIA WOMEN TOUR OF INDIA 2022,"BRABOURNE - CCI , MUMBAI",17 DEC 2022,7:00 PM IST
4,5th T20I,AUSTRALIA WOMEN TOUR OF INDIA 2022,"BRABOURNE - CCI , MUMBAI",20 DEC 2022,7:00 PM IST
5,2nd Test,INDIA TOUR OF BANGLADESH TEST SERIES 2022-23,"SHERE BANGLA NATIONAL STADIUM, MIRPUR , DHAKA",22 DEC 2022,9:30 AM IST
6,1st T20I,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,"WANKHEDE STADIUM , MUMBAI",3 JAN 2023,7:30 PM IST
7,2nd T20I,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,"MAHARASHTRA CRICKET ASSOCIATION STADIUM , PUNE",5 JAN 2023,7:30 PM IST
8,3rd T20I,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,"SAURASHTRA CRICKET ASSOCIATION STADIUM , RAJKOT",7 JAN 2023,7:30 PM IST
9,1st ODI,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,"BARSAPARA CRICKET STADIUM , GUWAHATI",10 JAN 2023,1:30 PM IST


In [5]:
'''
3. Scrape the details of selenium exception from guru99.com. Url = https://www.guru99.com/
You need to find following details:
A) Name
B) Description
Note: - From guru99 home page you have to reach to selenium exception handling page through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up guru99.com website on automated chrome window
url = 'https://www.guru99.com/'
driver.get(url)

# clicking on selenium
search_btn = driver.find_element(By.XPATH, "//a[contains(text(),'➤ Selenium')]")
search_btn.click()
time.sleep(2)

# clicking on selenium exception handling
search_btn = driver.find_element(By.XPATH, "//a[@title='Selenium Exception Handling (Common Exceptions List)']")
search_btn.click()
time.sleep(2)

# initializing required empty list
name = []
description = []

for i in range(0,41):
    name_tag = driver.find_element(By.XPATH,"//div[@class='entry-content single-content']/p[{}]".format(str(i+1)))
    name.append(name_tag.text.split(':')[0].strip())


    description_tags = driver.find_element(By.XPATH,"//div[@class='entry-content single-content']/p[{}]".format(str(i+1)))
    description.append(description_tags.text.split(':')[1].strip())

df = pd.DataFrame({'Name':name, 'Description':description})
df

,Name,Description
0,1. ElementNotVisibleException,This type of Selenium exception occurs when an...
1,2. ElementNotSelectableException,This Selenium exception occurs when an element...
2,3. NoSuchElementException,This Exception occurs if an element could not ...
3,4. NoSuchFrameException,This Exception occurs if the frame target to b...
4,5. NoAlertPresentException,This Exception occurs when you switch to no pr...
5,6. NoSuchWindowException,This Exception occurs if the window target to ...
6,7. StaleElementReferenceException,This Selenium exception occurs happens when th...
7,8. SessionNotFoundException,The WebDriver is acting after you quit the bro...
8,9. TimeoutException,Thrown when there is not enough time for a com...
9,10. WebDriverException,This Exception takes place when the WebDriver ...


In [6]:
'''
4. Scrape the details of State-wise GDP of India from statisticstime.com. Url = http://statisticstimes.com/
You have to find following details:
A) Rank
B) State
C) GSDP(18-19)- at current prices
D) GSDP(19-20)- at current prices
E) Share(18-19)
F) GDP($ billion)
Note: - From statisticstimes home page you have to reach to economy page through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up statisticstimes.com website on automated chrome window
url = 'http://statisticstimes.com/'
driver.get(url)

# clicking on economy
search_btn = driver.find_element(By.XPATH, "//button[normalize-space()='Economy']")
search_btn.click()
time.sleep(1)

# clicking on india
search_btn = driver.find_element(By.XPATH, "//a[@href='economy/india-statistics.php']")
search_btn.click()
time.sleep(2)

# to close add if it appears
try:
    # switching to the frame of ad
    driver.switch_to.frame("aswift_2")
    driver.switch_to.frame("ad_iframe")

    search_btn = driver.find_element(By.XPATH, '//*[@id="dismiss-button"]')
    search_btn.click()
    time.sleep(2)
    driver.switch_to.parent_frame()
    driver.switch_to.parent_frame()
except:
    pass

# clicking on GDP of Indian states
search_btn = driver.find_element(By.XPATH, "//a[normalize-space()='» GDP of Indian states']")
search_btn.click()
time.sleep(2)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_results(self):
          
        columns = ['Rank','State','GSDP (Cr INR at Current prices) [19-20]','GSDP (Cr INR at Current prices) [18-19]','Share','GDP($ billion)']
        data = {}
        time.sleep(2)

        elements = self.driver.find_elements(By.XPATH,"//table[@id='table_id']//tbody/tr")
        for element in elements:
            current_index = elements.index(element) + 1
            parsed_data = []
            for column in columns:
                value = element.find_element(By.XPATH,"//table[@id='table_id']//tbody/tr[{}]/td[{}]".format(current_index, columns.index(column) + 1)).text
                parsed_data.append(value)
            data.update({current_index: parsed_data})
        data = pd.DataFrame(data.values())
        data.columns = columns
        return data


if "__main__" == __name__:
    
    table = Table(driver)
    gdp = table.get_results()

In [7]:
gdp

,Rank,State,GSDP (Cr INR at Current prices) [19-20],GSDP (Cr INR at Current prices) [18-19],Share,GDP($ billion)
0,1,Maharashtra,-,"2,632,792",13.94%,399.921
1,2,Tamil Nadu,"1,845,853","1,630,208",8.63%,247.629
2,3,Uttar Pradesh,"1,687,818","1,584,764",8.39%,240.726
3,4,Gujarat,-,"1,502,899",7.96%,228.290
4,5,Karnataka,"1,631,977","1,493,127",7.91%,226.806
5,6,West Bengal,"1,253,832","1,089,898",5.77%,165.556
6,7,Rajasthan,"1,020,989","942,586",4.99%,143.179
7,8,Andhra Pradesh,"972,782","862,957",4.57%,131.083
8,9,Telangana,"969,604","861,031",4.56%,130.791
9,10,Madhya Pradesh,"906,672","809,592",4.29%,122.977


In [9]:
'''
5. Scrape the details of trending repositories on Github.com. Url = https://github.com/
You have to find the following details:
A) Repository title
B) Repository description
C) Contributors count
D) Language used
Note: - From the home page you have to click on the trending option from Explore menu through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up github.com website on automated chrome window
url = 'https://github.com/'
driver.get(url)

# clicking on open source
search_btn = driver.find_element(By.XPATH, "//button[normalize-space()='Open Source']")
search_btn.click()
time.sleep(1)

# clicking on trending
search_btn = driver.find_element(By.XPATH, "//a[normalize-space()='Trending']")
search_btn.click()
time.sleep(2)

# initializing required empty list
repo_title = []
repo_description = []
contributors_count = []
language_count = []
urls = []

delay = 5
cycle = 0

# for storing repository title:
repo_title_tags = driver.find_elements(By.XPATH,"//article/h1")
for i in repo_title_tags:
    repo_title.append(i.text)

url_tag = driver.find_elements(By.XPATH,"//article/h1/a")
for i in url_tag:
    urls.append(i.get_attribute('href'))

for url in urls:
    driver.get(url)
    time.sleep(2)
    
    cycle += 1
    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, "//p[@class='f4 my-3']")))
        repo_description_tag = driver.find_element(By.XPATH,"//p[@class='f4 my-3']")
        repo_description.append(repo_description_tag.text)
        
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH,"//a[contains(text(),'Contributors')]/span")))
        contributors_count_tag = driver.find_element(By.XPATH,"//a[contains(text(),'Contributors')]/span")
        contributors_count.append(contributors_count_tag.text)
        
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH,'//div[@class="BorderGrid-row"]/div[1]/ul[@class="list-style-none"]')))
        language_count_tags = driver.find_element(By.XPATH,'//div[@class="BorderGrid-row"]/div[1]/ul[@class="list-style-none"]')
        language_count.append(language_count_tags.text.replace('\n',' '))
        
    except:
        if len(repo_description) != cycle:
            repo_description.append('-')
            
        if len(contributors_count) != cycle:
            contributors_count.append('-')
        
        if len(language_count) != cycle:
            language_count.append('-')

df = pd.DataFrame({'Repository title':repo_title, 'Repository description':repo_description, 'Contributors count':contributors_count, 'Language used':language_count})
df

,Repository title,Repository description,Contributors count,Language used
0,fuergaosi233 / wechat-chatgpt,Use ChatGPT On Wechat via wechaty,21,TypeScript 88.5% Python 8.3% Dockerfile 2.6% S...
1,869413421 / wechatbot,为个人微信接入ChatGPT,4,Go 89.4% Dockerfile 9.6% Makefile 1.0%
2,exaloop / codon,"A high-performance, zero-overhead, extensible ...",4,C++ 55.2% Python 43.4% Other 1.4%
3,doocs / leetcode,😏 LeetCode solutions in any programming langua...,108,
4,teaxyz / cli,the unified package manager (brew2),17,TypeScript 100.0%
5,Tencent / Hippy,Hippy is designed to easily build cross-platfo...,80,android-release linux-release com.tencent.hipp...
6,cloneofsimo / lora,Using Low-rank adaptation to quickly fine-tune...,2,Jupyter Notebook 99.1% Other 0.9%
7,AutumnWhj / ChatGPT-wechat-bot,ChatGPT for wechat,2,TypeScript 57.0% JavaScript 29.3% Dockerfile 1...
8,PathOfBuildingCommunity / PathOfBuilding,Offline build planner for Path of Exile.,187,Lua 100.0%
9,surrealdb / surrealdb,"A scalable, distributed, collaborative, docume...",31,Rust 98.9% Other 1.1%


In [10]:
'''
6. Scrape the details of top 100 songs on billiboard.com. Url = https:/www.billboard.com/
You have to find the following details:
A) Song name
B) Artist name
C) Last week rank
D) Peak rank
E) Weeks on board
Note: - From the home page you have to click on the charts option then hot 100-page link through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up billboard.com website on automated chrome window
url = 'https:/www.billboard.com/'
driver.get(url)

# clicking on menu
search_btn = driver.find_element(By.XPATH, "//div[contains(@class,'lrv-u-flex lrv-u-justify-content-start lrv-u-align-items-center')]/button//*[name()='svg']")
search_btn.click()
time.sleep(1)

# clicking on Hot 100 under Charts
search_btn = driver.find_element(By.XPATH, "//a[normalize-space()='Hot 100']")
search_btn.click()
time.sleep(3)

# initializing required empty list
song_name = []
artist_name = []
last_week_rank = []
peak_rank = []
weeks_on_board = []

cycle = 0

rows = driver.find_elements(By.XPATH,'//div[@class="o-chart-results-list-row-container"]')

for i in rows:
    
    cycle += 1
    
    song_name_tag = driver.find_element(By.XPATH,'//div[@class="o-chart-results-list-row-container"][{}]/ul[1]/li[4]/ul/li[1]/h3'.format(str(cycle)))
    song_name.append(song_name_tag.text)


    artist_name_tags = driver.find_element(By.XPATH,'//div[@class="o-chart-results-list-row-container"][{}]/ul[1]/li[4]/ul/li[1]/span'.format(str(cycle)))
    artist_name.append(artist_name_tags.text)

    last_week_rank_tags = driver.find_element(By.XPATH,'//div[@class="o-chart-results-list-row-container"][{}]/ul[1]/li[4]/ul/li[4]'.format(str(cycle)))
    last_week_rank.append(last_week_rank_tags.text)
    
    peak_rank_tags = driver.find_element(By.XPATH,'//div[@class="o-chart-results-list-row-container"][{}]/ul[1]/li[4]/ul/li[5]'.format(str(cycle)))
    peak_rank.append(peak_rank_tags.text)

    weeks_on_board_tags = driver.find_element(By.XPATH,'//div[@class="o-chart-results-list-row-container"][{}]/ul[1]/li[4]/ul/li[6]'.format(str(cycle)))
    weeks_on_board.append(weeks_on_board_tags.text)

df = pd.DataFrame({'Song Name':song_name, 'Artist Name':artist_name, 'Last Week Rank':last_week_rank, 'Peak Rank':peak_rank, 'Weeks_on_board':weeks_on_board})
df

,Song Name,Artist Name,Last Week Rank,Peak Rank,Weeks_on_board
0,Anti-Hero,Taylor Swift,1,1,6
1,All I Want For Christmas Is You,Mariah Carey,5,1,54
2,Rockin' Around The Christmas Tree,Brenda Lee,6,2,48
3,Unholy,Sam Smith & Kim Petras,3,1,10
4,Jingle Bell Rock,Bobby Helms,9,3,45
...,...,...,...,...,...
95,Someday At Christmas,Lizzo,-,96,1
96,Vigilante Shit,Taylor Swift,73,10,6
97,Forget Me,Lewis Capaldi,-,95,6
98,"Going, Going, Gone",Luke Combs,98,98,3


In [ ]:
'''
7. Scrape the details of Data science recruiters from naukri.com. Url = https://www.naukri.com/
You have to find the following details:
A) Name
B) Designation
C) Company
D) Skills they hire for
E) Location
Note: - From naukri.com homepage click on the recruiters option and the on the search pane type Data science 
and click on search. All this should be done through code
'''
# Skipped as per Sir's direction.

In [12]:
'''
8. Scrape the details of Highest selling novels.
Url = https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-grey-compare/
You have to find the following details:
A) Book name
B) Author name
C) Volumes sold
D) Publisher
E) Genre
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up statisticstimes.com website on automated chrome window
url = 'https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-grey-compare/'
driver.get(url)
time.sleep(1)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_results(self):
          
        columns = ['Book Name','Author Name','Volume Sold','Publisher','Genre']
        data = {}
        time.sleep(2)

        elements = self.driver.find_elements(By.XPATH,'//div[@class="embed block"]//tbody/tr')
        for element in elements:
            current_index = elements.index(element) + 1
            parsed_data = []
            for column in columns:
                value = element.find_element(By.XPATH,'//div[@class="embed block"]//tbody/tr[{}]/td[{}]'.format(current_index, columns.index(column) + 2)).text
                parsed_data.append(value)
            data.update({current_index: parsed_data})
        data = pd.DataFrame(data.values())
        data.columns = columns
        return data


if "__main__" == __name__:
    
    table = Table(driver)
    highest_selling_novels = table.get_results()

In [13]:
highest_selling_novels

,Book Name,Author Name,Volume Sold,Publisher,Genre
0,"Da Vinci Code,The","Brown, Dan","5,094,805",Transworld,"Crime, Thriller & Adventure"
1,Harry Potter and the Deathly Hallows,"Rowling, J.K.","4,475,152",Bloomsbury,Children's Fiction
2,Harry Potter and the Philosopher's Stone,"Rowling, J.K.","4,200,654",Bloomsbury,Children's Fiction
3,Harry Potter and the Order of the Phoenix,"Rowling, J.K.","4,179,479",Bloomsbury,Children's Fiction
4,Fifty Shades of Grey,"James, E. L.","3,758,936",Random House,Romance & Sagas
...,...,...,...,...,...
95,"Ghost,The","Harris, Robert","807,311",Random House,General & Literary Fiction
96,Happy Days with the Naked Chef,"Oliver, Jamie","794,201",Penguin,Food & Drink: General
97,"Hunger Games,The:Hunger Games Trilogy","Collins, Suzanne","792,187",Scholastic Ltd.,Young Adult Fiction
98,"Lost Boy,The:A Foster Child's Search for the L...","Pelzer, Dave","791,507",Orion,Biography: General


In [14]:
'''
9. Scrape the details most watched tv series of all time from imdb.com. Url= https://www.imdb.com/list/ls095964455/
You have to find the following details: 
A) Name
B) Year span
C) Genre
D) Run time 
E) Ratings 
F) Votes
'''


def movies(x):
    page = requests.get(x)
    soup = BeautifulSoup(page.content)
    
    name = [] # empty list to store all movies
    for i in soup.find_all('h3', class_="lister-item-header"):
        name.append(i.text.split("\n")[2])
        
    year_span = [] # empty list to store all years
    for i in soup.find_all('span', class_="lister-item-year text-muted unbold"):
        year_span.append(i.text.split('(',)[1].split(')')[0])
        
    ratings = [] # empty list to store all ratings
    for i in soup.find_all('div', class_ = "ipl-rating-star small"):
        ratings.append(i.text.split('\n')[8])    
        
    genre = []
    for i in soup.find_all('span', class_ = "genre"):
        genre.append(i.text.replace('\n', ''))
    
    runtime = []
    for i in soup.find_all('span', class_ = "runtime"):
        runtime.append(i.text)
    
    votes = []
    for i in soup.find_all('div', class_ = "lister-item-content"):
        votes.append(i.text.split('Votes:')[1].replace('\n',''))
        
    df = pd.DataFrame({'Name':name, 'Year Span':year_span, 'Genre':genre, 'Run Time':runtime, 'Ratings':ratings, 'Votes':votes})
    df
    return (df)

# calling the function using link as the parameter
movies('https://www.imdb.com/list/ls095964455/')

,Name,Year Span,Genre,Run Time,Ratings,Votes
0,Game of Thrones,2011–2019,"Action, Adventure, Drama",57 min,9.2,"2,091,231"
1,Stranger Things,2016–,"Drama, Fantasy, Horror",51 min,8.7,"1,180,712"
2,The Walking Dead,2010–2022,"Drama, Horror, Thriller",44 min,8.1,"989,594"
3,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",60 min,7.5,"292,983"
4,The 100,2014–2020,"Drama, Mystery, Sci-Fi",43 min,7.6,"252,155"
...,...,...,...,...,...,...
95,Reign,2013–2017,Drama,42 min,7.4,"50,207"
96,A Series of Unfortunate Events,2017–2019,"Adventure, Comedy, Drama",50 min,7.8,"61,517"
97,Criminal Minds,2005–,"Crime, Drama, Mystery",42 min,8.1,"198,519"
98,Scream,2015–2019,"Comedy, Crime, Drama",45 min,7.1,"41,591"


In [15]:
'''
10. Details of Datasets from UCI machine learning repositories. Url = https://archive.ics.uci.edu/
You have to find the following details:
A) Dataset name
B) Data type
C) Task
D) Attribute type
E) No of instances
F) No of attribute
G) Year
Note: - from the home page you have to go to the ShowAllDataset page through code.
'''
# Connect to web driver
driver = webdriver.Chrome(r"chromedriver")

# Opening up statisticstimes.com website on automated chrome window
url = 'https://archive.ics.uci.edu/'
driver.get(url)

button = driver.find_element(By.XPATH, '//b[normalize-space()="View ALL Data Sets"]')
button.click()
time.sleep(2)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_results(self):
          
        columns = ['Dataset Name','Data Type','Task','Attribute Type','No. of Instances', 'No. of attribute', 'Year']
        data = {}
        time.sleep(2)

        elements = self.driver.find_elements(By.XPATH,'//table[@border="1"]/tbody/tr')
        for element in elements:
            current_index = elements.index(element) + 1
            parsed_data = []
            for column in columns:
                value = element.find_element(By.XPATH,'//table[@border="1"]/tbody/tr[{}]/td[{}]'.format(current_index, columns.index(column) + 1)).text
                parsed_data.append(value)
            data.update({current_index: parsed_data})
        data = pd.DataFrame(data.values())
        data.columns = columns
        return data.drop(0)


if "__main__" == __name__:
    
    table = Table(driver)
    datasets_uci = table.get_results()

In [16]:
datasets_uci

,Dataset Name,Data Type,Task,Attribute Type,No. of Instances,No. of attribute,Year
1,Abalone,Multivariate,Classification,"Categorical, Integer, Real",4177,8,1995
2,Adult,Multivariate,Classification,"Categorical, Integer",48842,14,1996
3,Annealing,Multivariate,Classification,"Categorical, Integer, Real",798,38,
4,Anonymous Microsoft Web Data,,Recommender-Systems,Categorical,37711,294,1998
5,Arrhythmia,Multivariate,Classification,"Categorical, Integer, Real",452,279,1998
...,...,...,...,...,...,...,...
618,Influenza outbreak event prediction via Twit...,Multivariate,Classification,"Integer, Real",75840,525,2020
619,Turkish Music Emotion Dataset,Multivariate,Classification,"Integer, Real",400,50,2020
620,Maternal Health Risk Data Set,,Classification,,1014,7,2020
621,Room Occupancy Estimation,"Multivariate, Time-Series",Classification,Real,10129,16,2021
